# 含并⾏连结的⽹络
在2014年的ImageNet图像识别挑战赛中，⼀个名叫GoogLeNet的⽹络结构⼤放异彩 [1]。它虽然
在名字上向LeNet致敬，但在⽹络结构上已经很难看到LeNet的影⼦。 GoogLeNet吸收了NiN中⽹
络串联⽹络的思想，并在此基础上做了很⼤改进。在随后的⼏年⾥，研究⼈员对GoogLeNet进⾏
了数次改进，本节将介绍这个模型系列的第⼀个版本。
## Inception块
GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上
⼀节介绍的NiN块相⽐，这个基础块在结构上更加复杂，如图5.8所⽰

![Inception块的结构](../img/inception.svg)

由图5.8可以看出， Inception块⾥有4条并⾏的线路。前3条线路使⽤窗口⼤小分别是1 × 1、 3 ×
3和5 × 5的卷积层来抽取不同空间尺⼨下的信息，其中中间2个线路会对输⼊先做1 × 1卷积来减
少输⼊通道数，以降低模型复杂度。第四条线路则使⽤3 × 3最⼤池化层，后接1 × 1卷积层来改变
通道数。 4条线路都使⽤了合适的填充来使输⼊与输出的⾼和宽⼀致。最后我们将每条线路的输
出在通道维上连结，并输⼊接下来的层中去。

Inception块中可以⾃定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [1]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

class Inception(nn.Block):
	# c1 - c4为每条线路里的层的输出通道数
	def __init__(self, c1, c2, c3, c4, **kwargs):
		super(Inception, self).__init__(**kwargs)
		# 线路1，单1 x 1卷积层
		self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
		# 线路2，1 x 1卷积层后接3 x 3卷积层
		self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
		self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')
		# 线路3，1 x 1卷积层后接5 x 5卷积层
		self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
		self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')
		# 线路4，3 x 3最大池化层后接1 x 1卷积层
		self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
		self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')

	def forward(self, x):
		p1 = self.p1_1(x)
		p2 = self.p2_2(self.p2_1(x))
		p3 = self.p3_2(self.p3_1(x))
		p4 = self.p4_2(self.p4_1(x))
		return nd.concat(p1, p2, p3, p4, dim=1)  # 在通道维上连结输出

## GoogLeNet模型
GoogLeNet跟VGG⼀样，在主体卷积部分中使⽤5个模块（block），每个模块之间使⽤步幅为2的3×
3最⼤池化层来减小输出⾼宽。第⼀模块使⽤⼀个64通道的7 × 7卷积层。

In [2]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2,padding=1))

第⼆模块使⽤2个卷积层：⾸先是64通道的1 × 1卷积层，然后是将通道增⼤3倍的3 × 3卷积层。它
对应Inception块中的第⼆条线路。

In [3]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
nn.MaxPool2D(pool_size=3, strides=2, padding=1))
